In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import shutil

In [ ]:
!pip install dirsync
import dirsync

In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
base = "/content/drive/MyDrive/cars/train"
dest_base = '/content/drive/MyDrive/data/train'
IMAGE_SIZE = (256, 256)

In [ ]:
augment = tf.keras.Sequential([
            tf.keras.layers.RandomFlip(mode='horizontal'),
            tf.keras.layers.RandomTranslation((-.1, .1),
                                      (-.1, .1),
                                      fill_mode='constant'),
            ])

In [ ]:
def augmentor(base, color, rate=2):
    file_counter = 1000
    path = os.path.join(base, color)
    os.makedirs(os.path.join(dest_base, color), exist_ok=True)
    move_originals(path, os.path.join(dest_base, color))
    ds = get_data(path)
    for _ in range(rate):
        augmented = augment(ds)
        for image in augmented:
            img = Image.fromarray(image.numpy().astype(np.uint8))
            img.save(os.path.join(dest_base, color, str(file_counter)+".jpg"))
            file_counter += 1

In [ ]:
def get_data(path: str):
    images = np.array([]).reshape(-1, *IMAGE_SIZE, 3)
    files = [os.path.join(path, file) 
             for file in os.listdir(path)]
    for file in files:
        image = Image.open(file).resize(IMAGE_SIZE)
        image = np.expand_dims(image, 0)
        images = np.concatenate([images, image], axis=0)
    return images

In [ ]:
def move_originals(path, dest):
  for file in os.listdir(path):
    shutil.copyfile(os.path.join(path, file), os.path.join(dest, file))

In [ ]:
to_augment = [("cream", 2), ("crimson", 3), ("gold", 2), ("navy-blue", 2), ("titanium", 2)]
for color, rate in to_augment:
    augmentor(base, color=color, rate=rate)

In [ ]:
dirsync.sync("/content/drive/MyDrive/cars/val",
             "/content/drive/MyDrive/data/val",
             action="sync")

In [ ]:
remain_colors = [color 
                 for color in os.listdir(base)
                 if color not in os.listdir("/content/drive/MyDrive/data/train")]

In [ ]:
for color in remain_colors:
  src_dir = os.path.join("/content/drive/MyDrive/cars/train", color)
  dst_dir = os.path.join("/content/drive/MyDrive/data/train", color) 
  os.makedirs(dst_dir, exist_ok=True)
  for file in os.listdir(src_dir):
    shutil.copyfile(os.path.join(src_dir, file), os.path.join(dst_dir, file))
  print(color, "is completed")